In [ ]:
'''
All the former DNA code from notebook 16-Adapting biomass coefficients to fit experimental data

#Parse the fasta file
os.chdir('/home/jean-christophe/Documents/Maitrise_UCSD/GenBank_files/')
genome = SeqIO.read('Mflorum_DNA.fasta','fasta')

#Get the number of each letter in the genome
base_genome = {'A':0,'T':0,'C':0,'G':0}
for element in genome:
    value = base_genome.get(element)
    new_value = value + 1
    base_genome[element] = new_value    
    
#Get the ratios for each letter in the genome
bases = ['A','T','C','G']
ratio_genome = {'A':0,'T':0,'C':0,'G':0}

for letter in bases:
    number_of_base = float(base_genome.get(letter))
    total = float(len(genome))
    ratio = number_of_base/total
    ratio_genome[letter]=ratio
    
    
#Transform the ratios into mmol/gDW
DNA_RATIO = 0.0839
DNA_WEIGHT = CELL_WEIGHT*DNA_RATIO

base_to_bigg = {'A':Mflorum.metabolites.datp_c,'T':Mflorum.metabolites.dttp_c,
                'C':Mflorum.metabolites.dctp_c,'G':Mflorum.metabolites.dgtp_c}
DNA_biomass_ratios = {'A':0,'T':0,'C':0,'G':0}
#Get the total weight of each letter
for letter in bases:
    ratio = ratio_genome.get(letter)
    total_weight = ratio*DNA_WEIGHT
    metab = base_to_bigg.get(letter)
    mol_weight = metab.formula_weight
    mmols_per_cell = (total_weight/mol_weight)*1000
    mmols_per_gDW = mmols_per_cell/CELL_WEIGHT
    DNA_biomass_ratios[letter] = mmols_per_gDW
    
    
#Change the ratios based on their appearance in the reactants and products of biomass eq

Algorythm should go like this:
1- Remove the metabolites from the BIOMASS equation
2- Add them with the new coefficients

#1- Remove the metabolites from the BIOMASS equation
remove_DNA_metab = {Mflorum.metabolites.datp_c:1.,Mflorum.metabolites.dttp_c:1.,
                    Mflorum.metabolites.dctp_c:1.,Mflorum.metabolites.dgtp_c:1.}
Mflorum.reactions.BIOMASS.add_metabolites(remove_DNA_metab)
Mflorum.reactions.BIOMASS.reaction

'''

In [19]:
'''
A class contains objects and methods
Objects and methods are defined within the class.
Objects and methods can be referenced outside the class
'''
class dummy:
    element = 'stuff'  
    def printhello(element):
        print(element)

In [20]:
i = dummy.element
dummy.printhello(i)

stuff


In [31]:
#Figure out inheritance
class sub_dummy(dummy):
    new_element = 'new_stuff'
    def sub_print(new_element):
        dummy.printhello(new_element)

In [33]:
class dumb_dummy():
    dumb_element = 'dumb_stuff'
    def dumb_print(dumb_element):
        dummy.printhello(dumb_element)

In [34]:
dumb_dummy.dumb_print(i)

stuff


## Architecture of the package for DNA

### Objects
1- The DNA bases, a list of characters called bases
2- 

### Methods
1- Import the genome from fasta file: import_genome
2- Get the number of each base in the genome
3- Get the ratio of each base in the genome
4- 

In [59]:
'''
User should provide:
- A path to the fasta file of the genome
- The total dry cell weight
- The fraction of DNA in the cell

'''
class DNA():
    #Mandatory imports
    import cobra
    #from biomass import Update
    
    #Objects
    #none so far
    
    #Methods
    def get_coefficients(path_to_fasta,path_to_model,CELL_WEIGHT = 9.7, DNA_RATIO = 0.0839):
        BASES = ['A','T','C','G']
        #Imports the genome as a fasta file
        #PATH provided by user
        def import_genome(fasta):
            from Bio import SeqIO
            genome = SeqIO.read(fasta, 'fasta')
            return genome
        
        #Imports the model
        #PATH provided by user
        #Checks for format
        def import_model(path_to_model):
            #Determine format to import
            model_format = path_to_model.split('.')[-1]

            #Import model 2 formats possible so far json and xml
            if model_format == 'json':
                model = cobra.io.load_json_model(path_to_model)
            elif model_format == 'xml':
                model = cobra.io.read_sbml_model(path_to_model)
                
            return model
      
        def get_number_of_bases(genome):
            #Counts the number of each letter in the genome
            base_genome = {'A':0,'T':0,'C':0,'G':0}
            for element in genome:
                value = base_genome.get(element)
                new_value = value + 1
                base_genome[element] = new_value 
            
            return base_genome
                
        def get_ratio(base_genome,genome):
            #Get the ratios for each letter in the genome
            ratio_genome = {'A':0,'T':0,'C':0,'G':0}

            for letter in BASES:
                number_of_base = float(base_genome.get(letter))
                total = float(len(genome))
                ratio = number_of_base/total
                ratio_genome[letter]=ratio
                
            return ratio_genome

        def convert_to_coefficient(model,ratio_genome,CELL_WEIGHT,DNA_RATIO):
            #Transform the ratios into mmol/gDW
            DNA_WEIGHT = CELL_WEIGHT*DNA_RATIO
            
            base_to_bigg = {'A':model.metabolites.datp_c,'T':model.metabolites.dttp_c,
                            'C':model.metabolites.dctp_c,'G':model.metabolites.dgtp_c}
            
            DNA_coefficients = {'A':0,'T':0,'C':0,'G':0}
            #Calculate the biomass coefficient for each metabolite
            for letter in BASES:
                ratio = ratio_genome.get(letter)
                total_weight = ratio*DNA_WEIGHT
                metab = base_to_bigg.get(letter)
                mol_weight = metab.formula_weight
                mmols_per_cell = (total_weight/mol_weight)*1000
                mmols_per_gDW = mmols_per_cell/CELL_WEIGHT
                DNA_coefficients[letter] = mmols_per_gDW
                
            return DNA_coefficients
        
        #Get the biomass coefficients
        genome = import_genome(path_to_fasta)
        base_in_genome = get_number_of_bases(genome)
        ratio_in_genome = get_ratio(base_in_genome,genome)
        biomass_coefficients = convert_to_coefficient(import_model(path_to_model),ratio_in_genome,CELL_WEIGHT,DNA_RATIO)
        return biomass_coefficients

In [60]:
DNA.get_coefficients('Mflorum.fna','finalMeso.json',CELL_WEIGHT=15.,DNA_RATIO=0.12)

{'A': 0.08967758656924092,
 'C': 0.03519460423442598,
 'G': 0.032048301783382235,
 'T': 0.0917917570981278}

In [ ]:
class Update():
    #Mandatory imports
    import cobra
    
    #Objects
    BASES = ['A','T','C','G']
    
    #Methods
    def 

In [ ]:
class Protein():
    

In [ ]:
class RNA():
    

In [22]:
import cobra
import os
os.getcwd()

'/Users/JC/Documents/Maitrise_UCSD/Functionnal_testing/biomass'

In [23]:
model = cobra.io.read_sbml_model('/Users/JC/Documents/Maitrise_UCSD/Functionnal_testing/iJO1366.xml')

In [24]:
metab = model.metabolites.datp_c

In [25]:
metab

<Metabolite datp_c at 0x104fcc9e8>

In [26]:
model.add_metabolites(metab)